# Entrenando un Modelo Markoviano Latente (HMM)

## Corpus de español:

* AnCora | Github: https://github.com/UniversalDependencies/UD_Spanish-AnCora

* usamos el conllu parser para leer el corpus: https://pypi.org/project/conllu/

* Etiquetas Universal POS (Documentación): https://universaldependencies.org/u/pos/

In [1]:
#@title dependencias previas
!pip install conllu
!git clone https://github.com/UniversalDependencies/UD_Spanish-AnCora.git

Cloning into 'UD_Spanish-AnCora'...
remote: Enumerating objects: 1371, done.
remote: Counting objects: 100% (387/387), done.
remote: Compressing objects: 100% (92/92), done.
remote: Total 1371 (delta 301), reused 381 (delta 295), pack-reused 984
Receiving objects: 100% (1371/1371), 434.03 MiB | 24.13 MiB/s, done.
Resolving deltas: 100% (994/994), done.


In [2]:
#@title leyendo el corpus AnCora
from conllu import parse_incr #Lectura dek archivo
wordList = [] #Crear una lista vacia
data_file = open("UD_Spanish-AnCora/es_ancora-ud-dev.conllu", "r", encoding="utf-8") #Selecciona el archivo en el datafile
for tokenlist in parse_incr(data_file): #Por cada lista de tokens el resultado. Parse para generar lista de tokens del resultado del datafile
    print(tokenlist.serialize()) #imprime la lista de tokens serializada (de manera estetica)

Se truncaron las últimas líneas 5000 del resultado de transmisión.
36	vez	vez	NOUN	ncfs000	Gender=Fem|Number=Sing	33	obl	33:obl	SpaceAfter=No|ArgTem=argM:adv
37	,	,	PUNCT	fc	PunctType=Comm	36	punct	36:punct	_
38	17	17	NUM	_	NumForm=Digit|NumType=Card	40	nummod	40:nummod	Entity=(CESSCASTP20011002160c16--3-gstype:gen
39	nuevos	nuevo	ADJ	aq0mp0	Gender=Masc|Number=Plur	40	amod	40:amod	_
40	fichajes	fichaje	NOUN	ncmp000	Gender=Masc|Number=Plur	33	obj	33:obj	SpaceAfter=No|ArgTem=arg1:pat
41	,	,	PUNCT	fc	PunctType=Comm	45	punct	45:punct	_
42	entre	entre	ADP	sps00	_	43	case	43:case	_
43	ellos	él	PRON	pp3mp000	Case=Acc,Nom|Gender=Masc|Number=Plur|Person=3|PronType=Prs	45	nmod	45:nmod	Entity=(CESSCASTP20011002160c16--1-CorefType:ident,gstype:gen)
44	el	el	DET	da0ms0	Definite=Def|Gender=Masc|Number=Sing|PronType=Art	45	det	45:det	Entity=(NOCOREF:Spec.person-person-2-gstype:spec
45	portugués	portugués	NOUN	ncms000	Gender=Masc|Number=Sing	40	appos	40:appos	_
46	Conceiçao	Conceiçao	PROPN	np00000	_	4

In [ ]:
#@title Estructura de los tokens etiquetados del corpus
tokenlist[1] #imprime el primer token de la lista (De la iteración que hicimos)

Token([('id', 2),
       ('form', 'cierto'),
       ('lemma', 'cierto'),
       ('upos', 'ADJ'),
       ('xpos', 'ADJ'),
       ('feats', OrderedDict([('Gender', 'Masc'), ('Number', 'Sing')])),
       ('head', 3),
       ('deprel', 'nsubj'),
       ('deps', None),
       ('misc', None)])

In [3]:
tokenlist[1]['form']+'|'+tokenlist[1]['upos'] #Del token list dame el form junto a la categoria gramatical (igual que la formula de dado)

'El|DET'

## Entrenamiento del modelo - Calculo de conteos:

* tags (tags) `tagCountDict`: $C(tag)$
* emisiones (word|tag) `emissionProbDict`: $C(word|tag)$
* transiciones (tag|prevtag) `transitionDict`: $C(tag|prevtag)$

In [ ]:
tagCountDict = {} #Cuantas veces aparece esa etiqueta
emissionDict = {} #Cuantas veces dada una etiqueta corresponde cierta palabra
transitionDict = {} #Dada una etiqueta previa cual le corresponde una subsiguiente

tagtype = 'upos' #La convención de etiquetas universal upos
data_file = open("UD_Spanish-AnCora/es_ancora-ud-dev.conllu", "r", encoding="utf-8")

# Calculando conteos (pre-probabilidades)
for tokenlist in parse_incr(data_file): #Para recorrer ese corpus
  prevtag = None #Antes del primer token es nulo, no hay palabra anterior
  for token in tokenlist: #Leyendo token por token en la list

    # C(tag) cuantas veces aparece la etiqueta en el corpus
    tag = token[tagtype]
    if tag in tagCountDict.keys(): #A medida que se va iterando si el elemento ya esta en la lista
      tagCountDict[tag] += 1 #Se continua
    else:
      tagCountDict[tag] = 1 #En caso que no, se asigna que es la primera vez que aparece

    # C(word|tag) -> probabilidades emision
    wordtag = token['form'].lower()+'|'+token[tagtype] # (word|tag) la palabara en minuscula junto a la etiqueta gramatical
    if wordtag in emissionDict.keys(): #Se repite el proceso, si esta en la wortag continua sino asignala
      emissionDict[wordtag] = emissionDict[wordtag] + 1
    else:
      emissionDict[wordtag] = 1

    #  C(tag|tag_previo) -> probabilidades transición
    if prevtag is None: #Si el tag anterior es nulo, asignale el tag
      prevtag = tag
      continue #Salta la iteración
    transitiontags = tag+'|'+prevtag #La transición es el tag más el tag anterior
    if transitiontags in transitionDict.keys(): #Si la transición esta en la lista
      transitionDict[transitiontags] = transitionDict[transitiontags] + 1 #Se suma 1 a la transición
    else: #Si no esta en la lista
      transitionDict[transitiontags] = 1 #Se asigna 1 a la transición
    prevtag = tag #Se asigna el tag como el tag anterior

#transitionDict
#emissionDict
#tagCountDict

## Entrenamiento del modelo - calculo de probabilidades
* probabilidades de transición:
$$P(tag|prevtag) = \frac{C(prevtag, tag)}{C(prevtag)}$$

* probabilidades de emisión:
 $$P(word|tag) = \frac{C(word|tag)}{C(tag)}$$

In [6]:
transitionProbDict = {} # matriz A
emissionProbDict = {} # matriz B

# transition Probabilities
for key in transitionDict.keys(): #Recorrer todas las llaves de transición
  tag, prevtag = key.split('|') #Los va a separar para obtener el tag y su tag anterior (split)
  if tagCountDict[prevtag]>0: #Para cada etiqueta anterior revise si dio mayor de cero (Si aparecio)
    transitionProbDict[key] = transitionDict[key]/(tagCountDict[prevtag]) #Calcular la probabilidad de transición
  else:
    print(key)

# emission Probabilities
for key in emissionDict.keys(): #Recorrer todas las llaves de transición
  word, tag = key.split('|') #Los va a separar para obtener la palabra y su tag
  if emissionDict[key]>0: #Para cada etiqueta anterior revise si dio mayor de cero (Si aparecio)
    emissionProbDict[key] = emissionDict[key]/tagCountDict[tag] #Calcular la probabilidad de transición
  else:
    print(key)

transitionProbDict['ADJ|ADJ']
#emissionProbDict

0.030217452696978255

In [7]:
transitionProbDict

{'NOUN|DET': 0.7051234338171443,
 'PUNCT|NOUN': 0.21745022412175544,
 'ADP|PUNCT': 0.10090433127082342,
 'ADJ|ADP': 0.012436935351402088,
 'NOUN|ADJ': 0.21575826037842416,
 'ADP|NOUN': 0.32356926925883456,
 'SCONJ|ADP': 0.017834095975595446,
 'VERB|SCONJ': 0.09595070422535211,
 'VERB|VERB': 0.029147490685952224,
 'NUM|VERB': 0.020600482138943676,
 'NOUN|NUM': 0.5239130434782608,
 '_|NOUN': 0.05962681121651204,
 'ADP|_': 0.8613312202852615,
 'DET|ADP': 0.5141382142438109,
 'VERB|ADP': 0.057022175290390706,
 '_|VERB': 0.03550295857988166,
 'DET|NOUN': 0.014177004065464401,
 'CCONJ|NOUN': 0.04951527155217346,
 'ADJ|CCONJ': 0.08459422283356259,
 'ADP|ADJ': 0.24936458627506355,
 'NOUN|ADP': 0.19464977120732135,
 'ADJ|NOUN': 0.17043677681642866,
 'PUNCT|ADJ': 0.24795255577520475,
 'VERB|PUNCT': 0.06964937331429479,
 'ADP|VERB': 0.24479509094893712,
 'PRON|DET': 0.01786378861183476,
 'DET|PRON': 0.024761904761904763,
 'PROPN|DET': 0.12095273539263118,
 'PROPN|PROPN': 0.22665345553628932,
 'AD

## Guardar parámetros del modelo

In [8]:
import numpy as np
np.save('transitionHMM.npy', transitionProbDict) #Grabar cada propiedad en un archivo
np.save('emissionHMM.npy', emissionProbDict) #Grabar las propiedades en el archivo
transitionProbdict = np.load('transitionHMM.npy', allow_pickle='TRUE').item()
transitionProbDict['ADJ|ADJ']

0.030217452696978255